# Email自动翻译translate
## Version 2.0
### 读取CSV文件自动进行翻译

In [1]:
import requests
import random
import json
import os
from hashlib import md5
import pandas as pd
from urllib import parse


In [2]:
base_dir='./email'
questionsfile='newdata.csv'
f = open(os.path.join(base_dir, questionsfile), encoding="utf-8")
questionsdf=pd.read_csv(f)
# 第0列为序号无意义，删除第0列unnamed
questionsdf.drop(questionsdf.columns[0],axis=1,inplace=True)
questionsdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Subject  580 non-null    object
 1   Body     576 non-null    object
 2   Content  580 non-null    object
dtypes: object(3)
memory usage: 13.7+ KB


# 百度翻译仅作为例子，无法实现高速翻译，所以需要你申请微软的翻译key

In [7]:
# Set 百度翻译api的参数，appid/appkey等.
appid = '20200323000403030'
appkey = 'lvm1XikOF8TxxwA2B6j2'
endpoint = 'http://api.fanyi.baidu.com'
path = '/api/trans/vip/translate'
url = endpoint + path

In [8]:
# baidu translate function
def make_md5(s, encoding='utf-8'):
    return md5(s.encode(encoding)).hexdigest()
def translate_baidu(text, to_lang, from_lang='auto'):
    # Generate salt and sign
    text=parse.quote(text)
    salt = random.randint(32768, 65536)
    sign = make_md5(appid + text + str(salt) + appkey)

    # Build request
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    payload = {'appid': appid, 'q': text, 'from': from_lang, 'to': to_lang, 'salt': salt, 'sign': sign}
    # Send request
    r = requests.post(url, params=payload, headers=headers)
    result=r.json()
    # 百度免费文本翻译服务不允许高频调用，一次这里休眠5秒
    import time
    time.sleep(1)
 
    return parse.unquote(result['trans_result'][0]['dst'])

## 百度翻译的例子

In [5]:
# 翻译的例子
text = newdf['Content'][1]
# 单次翻译文本长度限定为6000字节以内,每月的免费额度50000字符
print('source:',text)
# For list of language codes, please refer to `https://api.fanyi.baidu.com/doc/21`
to_lang =  'en'
result=translate_baidu(text,to_lang)
# 输出翻译结果
print('translate:',result)

NameError: name 'newdf' is not defined

## 百度翻译的例子：对pandas的dataframe数据进行多行调用翻译

In [79]:
# 对标题列进行翻译的示例，由于我是用的是免费额度，高频率调用会返回如下错误
# {'error_code': '54003', 'error_msg': 'Invalid Access Limit'}
# 这里的例子是翻译前三行
questionsdf=questionsdf[0:3]
# 对每行的Subject列进行翻译
questionsdf.loc[:,'Subject'] = questionsdf['Subject'].apply(lambda row: translate_baidu(row,to_lang))
print(questionsdf['Subject'])

0                     Name does not appear on query
1            BIS Messaging Center (reply requested)
2    Report Portfolio Selection Object Id API error
Name: Subject, dtype: object
